# Carte des arrondissements

Test du chrolopleth plotly avec la géographie des arrondissements de montréal

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import json

MAP_FPATH = '../assets/maps/arrondissements_montreal.geojson'

with open(MAP_FPATH) as f:
    arrond = json.load(f)
    
print(arrond.keys())
print(arrond['type'])
print(arrond['crs'])
print(arrond['features'])

id = 'no_qr'
fig = go.Figure(
    go.Choroplethmap(
        geojson=arrond,
        featureidkey='properties.'+id,
        locations=[f['properties'][id] for f in arrond['features']],
        z=[f['properties'][id] for f in arrond['features']],
        hovertext=[f['properties']['nom_qr'] for f in arrond['features']],
        colorscale="Viridis",
        marker_opacity=0.5, marker_line_width=0))
fig.update_layout(map=dict(center=dict(lat=45.5517, lon=-73.7073), style="carto-positron", zoom=8.7))
fig.show()

# Districts électoraux de Montréal
Il manque tout ce qui n'est pas techniquement partie de la ville de montréal, e.g. Westmount

In [ ]:
import plotly.graph_objects as go
import json

MAP_FPATH = '../assets/maps/districts_montreal.geojson'

with open(MAP_FPATH) as f:
    districts = json.load(f)
    
print(districts.keys())
print(districts['type'])
print(districts['name'])
print(len(districts['features']))
print(districts['features'][0]['geometry'])
print(districts['features'][0].keys())
print(districts['features'][0]['properties'].keys())

id = 'id'
fig = go.Figure(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[f['properties'][id] for f in districts['features']],
    hovertext=[f['properties']['nom'] for f in districts['features']],
    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(
    map=dict(
        center=dict(lat=45.5517, lon=-73.7073), zoom=8.7))
fig.show()

# Superposition des arrondissements et circos

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import json

ARROND_FPATH = '../assets/maps/arrondissements_montreal.geojson'
CIRCO_FPATH = '../assets/maps/districts_montreal.geojson'


with open(ARROND_FPATH) as f:
    arrond = json.load(f)
with open(CIRCO_FPATH) as f:
    districts = json.load(f)    

# En bleu les arrondissements
id = 'no_qr'
fig = go.Figure(
    go.Choroplethmap(
        geojson=arrond,
        featureidkey='properties.'+id,
        locations=[f['properties'][id] for f in arrond['features']],
        z=[0]*len(arrond['features']),
        hovertext=[f['properties']['nom_qr'] for f in arrond['features']],
        marker_opacity=0.5, marker_line_width=5, marker_line_color='blue'))

# En rouge les circonscriptions
id = 'id'
fig = fig.add_trace(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[1]*len(districts['features']),
    hovertext=[f['properties']['nom'] for f in districts['features']],
    colorscale="Viridis",
    marker_opacity=0.5, marker_line_width=5, marker_line_color='red'))
fig.update_layout(
    showlegend=True,
    map=dict(
        center=dict(lat=45.5517, lon=-73.7073), zoom=8.7))
fig.show()

# Test des 125 districts électoraux du Québec

In [ ]:
import plotly.graph_objects as go
import numpy as np
import json

MAP_FPATH = '../assets/maps/districts_QC.geojson'

with open(MAP_FPATH) as f:
    districts = json.load(f)
    
districts['features'] = districts['features'][:1000]
for i in range(len(districts['features'])):
    districts['features'][i]['properties']['ID'] = i

print(districts.keys())
print(districts['type'])
print(districts['name'])
print(districts['crs'])
print(len(districts['features']))
print(districts['features'][0].keys())
for key in districts['features'][0]['properties'].keys():
    print('|\t', key, ':', districts['features'][0]['properties'][key])

id='ID'
fig = go.Figure(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[f['properties'][id] for f in districts['features']],
    hovertext=[f['properties']['NM_CEP'] for f in districts['features']],
    marker_opacity=0.5, marker_line_width=1))

fig.update_geos(
    projection=dict(
        type="conic conformal",
        parallels=[50, 46],
    ))
fig.update_layout(map=dict(center=dict(lat=47, lon=-70), zoom=4))

fig.show()

## Test du datasets de langues etc

Des données de langue sur l'ensemble du québec (enfin les villes principales), juste pour voir comment c'était organisé.

In [ ]:
import pandas as pd

FPATH =  '../assets/data/langues.csv'
df = pd.read_csv(FPATH, sep=';')

In [ ]:
# polars est pas mal plus rapide
import polars as pl

FPATH =  '../assets/data/langues.csv'
df = pl.read_csv(FPATH, separator=';')

# Dropping empty columns
df = df.drop(["Symbole"]+[f"Symbole_duplicated_{i}" for i in range(8)])


In [ ]:
print(*df.columns, sep='\n')

In [ ]:
print(df.head())

In [ ]:
print(*df['GÉO'].unique().to_list(), sep='\n')